In [ ]:
import pandas as pd
from mainnet_launch.constants import AutopoolConstants, AUTO_LRT, AUTO_USD


from mainnet_launch.data_fetching.get_state_by_block import (
    safe_normalize_6_with_bool_success,
    safe_normalize_with_bool_success,
    get_raw_state_by_blocks,
    get_state_by_one_block,
)

from mainnet_launch.database.schema.postgres_operations import (
    merge_tables_as_df,
    TableSelector,
    get_full_table_as_df,
    insert_avoid_conflicts,
)
from mainnet_launch.database.schema.full import *
from multicall import Call

from mainnet_launch.pages.rebalance_events.rebalance_events import _load_full_rebalance_event_df

ENGINE.echo = False

autopool = AUTO_USD

df = _load_full_rebalance_event_df(autopool)

destination_info_df = merge_tables_as_df(
    selectors=[
        TableSelector(
            AutopoolDestinations,
            [
                AutopoolDestinations.destination_vault_address,
                AutopoolDestinations.autopool_vault_address,
            ],
        ),
        TableSelector(
            table=DestinationTokens,
            select_fields=[
                DestinationTokens.token_address,
            ],
            join_on=DestinationTokens.destination_vault_address == AutopoolDestinations.destination_vault_address,
        ),
        TableSelector(
            table=Destinations,
            select_fields=[Destinations.pool],
            join_on=Destinations.destination_vault_address == AutopoolDestinations.destination_vault_address,
        ),
        TableSelector(
            table=Tokens,
            select_fields=[Tokens.decimals, Tokens.name],
            join_on=Tokens.token_address == DestinationTokens.token_address,
        ),
    ],
    where_clause=AutopoolDestinations.autopool_vault_address == autopool.autopool_eth_addr,
)

2025-06-02 12:20:09.062 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 12:20:09.063 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 12:20:09.064 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 12:20:09.065 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 12:20:09.065 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 12:20:09.066 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 12:20:09.066 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 12:20:09.066 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

,tx_hash,autopool_vault_address,chain_id,rebalance_file_path,destination_out,destination_in,quantity_out,quantity_in,safe_value_out,safe_value_in,spot_value_in,spot_value_out,swap_offset_period,block,datetime,destination_in_symbol,destination_out_symbol,destination_in_tokens,destination_out_tokens,tokens_move_name,spot_swap_cost,spot_slippage_bps,safe_swap_cost,safe_slippage_bps,file_name,datetime_generated,solver_address,rebalance_type,token_out,token_in,move_name,amount_out,amount_out_safe_value,min_amount_in,min_amount_in_safe_value,amount_out_spot_value,out_dest_apr,min_amount_in_spot_value,in_dest_apr,in_dest_adj_apr,apr_delta,num_candidate_destinations,candidate_destinations_rank,projected_swap_cost,projected_net_gain,projected_gross_gain,projected_slippage
0,0x67618958fa10c91e18218c3c3f7eb57509e70445e898...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743708477_0xa7569A44f348d3D70d...,0xb6E89d23d1E33537D383622B1955EbDa135c0b5A,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,85.899838,100.008658,100.118752,100.008658,100.008658,100.067556,None,22190481,2025-04-03 19:30:11+00:00,autoUSD,sUSDe,"('USDC',)","('sUSDe',)","('sUSDe',) -> ('USDC',)",0.058898,5.885779,0.110094,10.996327,rebalance_plan_1743708477_0xa7569A44f348d3D70d...,2025-04-03 19:27:57+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,ToIdle,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,sUSDe -> autoUSD,85.899838,1.001188e-10,99.888696,9.992867e-11,100.067556,0.061408,99.928668,0.061408,0.000000,-0.061408,0.0,NaN,1.900830e-13,1.900830e-13,0.000000,0.189858
1,0x5513250273bfdbf939553c9ed98ce48b8479bc739634...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743723845_0xa7569A44f348d3D70d...,0x6F628dcCD275feA4277722D177265741031E09d7,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,98.326351,99.954654,99.992786,99.954654,99.954654,99.980200,None,22191759,2025-04-03 23:46:35+00:00,autoUSD,crvUSDUSDT-f,"('USDC',)","('crvUSD', 'USDT')","('crvUSD', 'USDT') -> ('USDC',)",0.025546,2.555152,0.038132,3.813499,rebalance_plan_1743723845_0xa7569A44f348d3D70d...,2025-04-03 23:44:05+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,ToIdle,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,crvUSDUSDT-f -> autoUSD,98.326351,9.999279e-11,99.914672,9.995465e-11,99.980200,0.091285,99.954654,0.091285,0.000000,-0.091285,0.0,NaN,3.813300e-14,3.813300e-14,0.000000,0.038136
2,0x14425c16bdc198d43d738991130a7fea735c85d7c883...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743782101_0xa7569A44f348d3D70d...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,25503.391475,25348.990383,25503.391475,25505.925983,25502.655963,25503.391475,None,22196597,2025-04-04 15:57:35+00:00,crv2pool,autoUSD,"('USDT', 'USDC')","('USDC',)","('USDC',) -> ('USDT', 'USDC')",0.735512,0.288398,-2.534508,-0.993792,rebalance_plan_1743782101_0xa7569A44f348d3D70d...,2025-04-04 15:55:01+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,autoUSD -> crv2pool,25503.391475,2.550339e-08,25335.042437,2.550210e-08,25503.391475,0.000000,25498.822994,0.000000,0.090720,0.090720,12.0,0.0,1.287086e-12,7.140939e+01,71.409385,0.005047
3,0x5c3a705003f40a1e38309a79c873f54f147043dbcdd4...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743786510_0xa7569A44f348d3D70d...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x7583b1589aDD33320366A48A92794D77763FAE9e,19127.543600,18806.816895,19127.543600,19124.181930,19122.451703,19127.543600,None,22196965,2025-04-04 17:11:35+00:00,crvUSDUSDT-f,autoUSD,"('USDT', 'crvUSD')","('USDC',)","('USDC',) -> ('USDT', 'crvUSD')",5.091897,2.662076,3.361670,1.757502,rebalance_plan_1743786510_0xa7569A44f348d3D70d...,2025-04-04 17:08:30+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x390f3595bCa2Df7d23783dFd126427CCeb

In [ ]:
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_destination_token_values_tables import (
    _build_USD_autopool_price_calls,
)


def _get_spot_value_change_in_sovler(rebalance_event_row: dict) -> float:

    def _fetch_non_zero_changes(rebalance_event_row: dict) -> dict:
        tokens_and_decimals = destination_info_df[["token_address", "decimals"]].drop_duplicates()
        balance_of_calls = []

        decimal_normalizer_map = {
            6: safe_normalize_6_with_bool_success,
            18: safe_normalize_with_bool_success,
        }

        for _, token_row in tokens_and_decimals.iterrows():
            token = token_row["token_address"]
            normalizer = decimal_normalizer_map[token_row["decimals"]]
            call = Call(
                token,
                ["balanceOf(address)(uint256)", rebalance_event_row["solver_address"]],
                [(token, normalizer)],
            )
            balance_of_calls.append(call)

        balance_of_df = get_raw_state_by_blocks(
            balance_of_calls, [rebalance_event_row["block"] - 1, rebalance_event_row["block"]], chain=autopool.chain
        )
        diffs = balance_of_df.diff()
        second_row = diffs.iloc[1]
        non_zero_changes = {col: val for col, val in second_row.items() if val != 0}
        return non_zero_changes

    rebalance_event_row = df.iloc[-1]
    non_zero_changes = _fetch_non_zero_changes(rebalance_event_row)

    out_destination_sub_df = destination_info_df[
        destination_info_df["destination_vault_address"] == rebalance_event_row["destination_out"]
    ]
    in_destination_sub_df = destination_info_df[
        (destination_info_df["destination_vault_address"] == rebalance_event_row["destination_in"])
        & ~destination_info_df["token_address"].isin(out_destination_sub_df["token_address"])
    ]
    destination_token_in_for_spot_prices = pd.concat([out_destination_sub_df, in_destination_sub_df])

    spot_price_dict = get_state_by_one_block(
        _build_USD_autopool_price_calls(autopool.chain, destination_token_in_for_spot_prices),
        rebalance_event_row["block"],
        chain=autopool.chain,
    )
    token_spot_price = {t: value for (p, t, d), value in spot_price_dict.items()}
    spot_value_difference_in_solver = 0

    for k, v in non_zero_changes.items():
        spot_value_difference_in_solver += token_spot_price[k] * v

    # positve value means there was extra value left in solver
    # negative value means there was value taken out of the solver
    return spot_value_difference_in_solver


_get_spot_value_change_in_sovler(rebalance_event_row)

timestamp                                   2025-06-02 01:35:23+00:00  2025-06-02 01:35:35+00:00
0x83F20F44975D03b1b09e64809B757c47f942BEeA                   0.000000                   0.000000
0x0655977FEb2f289A4aB78af67BAB0d17aAb84367                   0.000000                   0.000000
0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E                   0.000000                   0.000000
0x853d955aCEf822Db058eb8505911ED77F175b99e                   0.000000                   0.000000
0x9D39A5DE30e57443BfF2A8307A4256c8797A3497                   8.064053                   8.064053
0xa3931d71877C0E7a3148CB7Eb4463524FEc27fbD                   0.000000                   0.000000
0xD4fa2D31b7968E448877f69A96DE69f5de8cD23E                   0.000000                   0.000000
0xdAC17F958D2ee523a2206206994597C13D831ec7                   0.000000                   0.000000
0x7Bc3485026Ac48b6cf9BaF0A377477Fff5703Af8                   0.000000                   0.000000
0xA0b86991c6218b36c1d19D4a2e9E

7.365628510823402

In [ ]:
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# ---------- assume these are imported from your codebase ----------
# from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_destination_token_values_tables import (
#     _build_USD_autopool_price_calls,
# )
# from your_module import get_raw_state_by_blocks, get_state_by_one_block, Call
# from your_module import safe_normalize_6_with_bool_success, safe_normalize_with_bool_success
# destination_info_df and autopool.chain should already exist


# ---------- fixed spot‑value function (doesn't override its input) ----------
def _get_spot_value_change_in_solver(rebalance_event_row: dict) -> float:
    def _fetch_non_zero_changes(row: dict) -> dict:
        tokens_and_decimals = destination_info_df[["token_address", "decimals"]].drop_duplicates()
        balance_of_calls = []
        decimal_normalizer_map = {
            6: safe_normalize_6_with_bool_success,
            18: safe_normalize_with_bool_success,
        }
        for _, token_row in tokens_and_decimals.iterrows():
            token = token_row["token_address"]
            normalizer = decimal_normalizer_map[token_row["decimals"]]
            call = Call(
                token,
                ["balanceOf(address)(uint256)", row["solver_address"]],
                [(token, normalizer)],
            )
            balance_of_calls.append(call)

        balance_of_df = get_raw_state_by_blocks(
            balance_of_calls,
            [row["block"] - 1, row["block"]],
            chain=autopool.chain,
        )
        diffs = balance_of_df.diff()
        second_row = diffs.iloc[1]
        return {col: val for col, val in second_row.items() if val != 0}

    non_zero_changes = _fetch_non_zero_changes(rebalance_event_row)
    out_dest = destination_info_df[
        destination_info_df["destination_vault_address"] == rebalance_event_row["destination_out"]
    ]
    in_dest = destination_info_df[
        (destination_info_df["destination_vault_address"] == rebalance_event_row["destination_in"])
        & ~destination_info_df["token_address"].isin(out_dest["token_address"])
    ]
    tokens_for_price = pd.concat([out_dest, in_dest])

    spot_price_dict = get_state_by_one_block(
        _build_USD_autopool_price_calls(autopool.chain, tokens_for_price),
        rebalance_event_row["block"],
        chain=autopool.chain,
    )
    token_spot_price = {t: value for (p, t, d), value in spot_price_dict.items()}

    spot_val_diff = 0
    for token, change in non_zero_changes.items():
        spot_val_diff += token_spot_price.get(token, 0) * change

    return spot_val_diff


# ---------- parallel execution with rate‑limit ----------
def compute_spot_changes(df: pd.DataFrame) -> list[float]:
    rows = df.to_dict("records")
    with ThreadPoolExecutor(max_workers=100) as executor:
        return list(executor.map(_get_spot_value_change_in_solver, rows))

    # ---------- usage example ----------


df["spot_value_difference_in_solver"] = compute_spot_changes(df)
df

,tx_hash,autopool_vault_address,chain_id,rebalance_file_path,destination_out,destination_in,quantity_out,quantity_in,safe_value_out,safe_value_in,spot_value_in,spot_value_out,swap_offset_period,block,datetime,destination_in_symbol,destination_out_symbol,destination_in_tokens,destination_out_tokens,tokens_move_name,spot_swap_cost,spot_slippage_bps,safe_swap_cost,safe_slippage_bps,file_name,datetime_generated,solver_address,rebalance_type,token_out,token_in,move_name,amount_out,amount_out_safe_value,min_amount_in,min_amount_in_safe_value,amount_out_spot_value,out_dest_apr,min_amount_in_spot_value,in_dest_apr,in_dest_adj_apr,apr_delta,num_candidate_destinations,candidate_destinations_rank,projected_swap_cost,projected_net_gain,projected_gross_gain,projected_slippage,spot_value_difference_in_solver,real_spot_swap_cost,real_spot_slippage
0,0x67618958fa10c91e18218c3c3f7eb57509e70445e898...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743708477_0xa7569A44f348d3D70d...,0xb6E89d23d1E33537D383622B1955EbDa135c0b5A,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,85.899838,100.008658,100.118752,100.008658,100.008658,100.067556,None,22190481,2025-04-03 19:30:11+00:00,autoUSD,sUSDe,"('USDC',)","('sUSDe',)","('sUSDe',) -> ('USDC',)",0.058898,5.885779,0.110094,10.996327,rebalance_plan_1743708477_0xa7569A44f348d3D70d...,2025-04-03 19:27:57+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,ToIdle,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,sUSDe -> autoUSD,85.899838,1.001188e-10,99.888696,9.992867e-11,100.067556,0.061408,99.928668,0.061408,0.000000,-0.061408,0.0,NaN,1.900830e-13,1.900830e-13,0.000000,0.189858,0.015010,NaN,NaN
1,0x5513250273bfdbf939553c9ed98ce48b8479bc739634...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743723845_0xa7569A44f348d3D70d...,0x6F628dcCD275feA4277722D177265741031E09d7,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,98.326351,99.954654,99.992786,99.954654,99.954654,99.980200,None,22191759,2025-04-03 23:46:35+00:00,autoUSD,crvUSDUSDT-f,"('USDC',)","('crvUSD', 'USDT')","('crvUSD', 'USDT') -> ('USDC',)",0.025546,2.555152,0.038132,3.813499,rebalance_plan_1743723845_0xa7569A44f348d3D70d...,2025-04-03 23:44:05+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,ToIdle,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,crvUSDUSDT-f -> autoUSD,98.326351,9.999279e-11,99.914672,9.995465e-11,99.980200,0.091285,99.954654,0.091285,0.000000,-0.091285,0.0,NaN,3.813300e-14,3.813300e-14,0.000000,0.038136,0.014997,NaN,NaN
2,0x14425c16bdc198d43d738991130a7fea735c85d7c883...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743782101_0xa7569A44f348d3D70d...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,25503.391475,25348.990383,25503.391475,25505.925983,25502.655963,25503.391475,None,22196597,2025-04-04 15:57:35+00:00,crv2pool,autoUSD,"('USDT', 'USDC')","('USDC',)","('USDC',) -> ('USDT', 'USDC')",0.735512,0.288398,-2.534508,-0.993792,rebalance_plan_1743782101_0xa7569A44f348d3D70d...,2025-04-04 15:55:01+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,autoUSD -> crv2pool,25503.391475,2.550339e-08,25335.042437,2.550210e-08,25503.391475,0.000000,25498.822994,0.000000,0.090720,0.090720,12.0,0.0,1.287086e-12,7.140939e+01,71.409385,0.005047,-0.008092,NaN,NaN
3,0x5c3a705003f40a1e38309a79c873f54f147043dbcdd4...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743786510_0xa7569A44f348d3D70d...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x7583b1589aDD33320366A48A92794D77763FAE9e,19127.543600,18806.816895,19127.543600,19124.181930,19122.451703,19127.543600,None,22196965,2025-04-04 17:11:35+00:00,crvUSDUSDT-f,autoUSD,"('USDT', 'crvUSD')","('USDC',)","('USDC',) -> ('USDT', 'crvUSD')",5.091897,2.662076,3.361670,1.757502,rebalance_plan_1743786510_0xa7569A44f348d3D70d...,2025-04-04 17:08:30+00:00,0xD02b50

In [122]:
df.columns

Index(['tx_hash', 'autopool_vault_address', 'chain_id', 'rebalance_file_path',
       'destination_out', 'destination_in', 'quantity_out', 'quantity_in',
       'safe_value_out', 'safe_value_in', 'spot_value_in', 'spot_value_out',
       'swap_offset_period', 'block', 'datetime', 'destination_in_symbol',
       'destination_out_symbol', 'destination_in_tokens',
       'destination_out_tokens', 'tokens_move_name', 'spot_swap_cost',
       'spot_slippage_bps', 'safe_swap_cost', 'safe_slippage_bps', 'file_name',
       'datetime_generated', 'solver_address', 'rebalance_type', 'token_out',
       'token_in', 'move_name', 'amount_out', 'amount_out_safe_value',
       'min_amount_in', 'min_amount_in_safe_value', 'amount_out_spot_value',
       'out_dest_apr', 'min_amount_in_spot_value', 'in_dest_apr',
       'in_dest_adj_apr', 'apr_delta', 'num_candidate_destinations',
       'candidate_destinations_rank', 'projected_swap_cost',
       'projected_net_gain', 'projected_gross_gain', 'projected

KeyError: "None of ['datetime'] are in the columns"

In [ ]:
df["real_spot_swap_cost"] = df["spot_swap_cost"] - df["spot_value_difference_in_solver"]
df["real_spot_slippage"] = 10000 * df["real_spot_swap_cost"] / df["spot_value_out"]

In [108]:
df.groupby("move_name")["real_spot_slippage"].describe().round(3)

,count,mean,std,min,25%,50%,75%,max
move_name,,,,,,,,
autoUSD -> GHOcrvUSD,24.0,2.957,2.085,0.274,1.251,2.623,3.943,7.632
autoUSD -> SkySaver,31.0,2.969,2.480,0.703,1.519,2.256,2.879,9.651
autoUSD -> crv2pool,60.0,5.334,5.784,-0.314,1.005,5.757,7.069,36.605
autoUSD -> crvU-sUSDe,16.0,1.315,3.129,-3.029,-1.024,0.916,2.931,7.172
autoUSD -> crvUSDUSDC-f,15.0,2.508,2.828,-0.101,0.582,1.097,4.057,7.617
autoUSD -> crvUSDUSDT-f,37.0,6.040,3.735,0.667,3.276,6.177,7.859,18.718
autoUSD -> crvUSDe,16.0,5.566,1.761,3.057,4.386,4.966,6.960,9.063
autoUSD -> fUSDC,29.0,-0.957,0.977,-3.274,-1.325,-0.778,-0.248,0.898
autoUSD -> fUSDT,29.0,3.171,2.083,0.658,1.234,2.280,5.315,5.903


In [152]:
px.bar(df["spot_value_difference_in_solver"])

In [149]:
px.line(df["spot_value_difference_in_solver"].cumsum())

In [125]:
df.groupby("move_name")["real_spot_swap_cost"].describe().round(3).sort_values("max")

,count,mean,std,min,25%,50%,75%,max
move_name,,,,,,,,
fUSDT -> crvU-sUSDe,1.0,-37.947,NaN,-37.947,-37.947,-37.947,-37.947,-37.947
crvUSDUSDT-f -> fUSDC,4.0,-45.495,9.842,-59.171,-49.283,-42.804,-39.017,-37.199
crv2pool -> crvUSDUSDC-f,1.0,-32.587,NaN,-32.587,-32.587,-32.587,-32.587,-32.587
autoUSD -> gtUSDCcore,1.0,-25.470,NaN,-25.470,-25.470,-25.470,-25.470,-25.470
sUSDe/USDC -> fUSDC,1.0,-16.822,NaN,-16.822,-16.822,-16.822,-16.822,-16.822
gtUSDC -> SkySaver,1.0,-4.843,NaN,-4.843,-4.843,-4.843,-4.843,-4.843
sUSDe/USDC -> frxsUSDS,1.0,-4.669,NaN,-4.669,-4.669,-4.669,-4.669,-4.669
crvUSDUSDT-f -> autoUSD,1.0,0.011,NaN,0.011,0.011,0.011,0.011,0.011
steakUSDC -> fUSDC,2.0,0.029,0.001,0.028,0.029,0.029,0.029,0.030


In [127]:
px.bar(df["spot_value_difference_in_solver"])

In [ ]:
def _fetch_some_non_zero_changes(rebalance_event_row: dict) -> float:
    tokens_and_decimals = destination_info_df[["token_address", "decimals"]].drop_duplicates()
    balance_of_calls = []
    decimal_normalizer_map = {
        6: safe_normalize_6_with_bool_success,
        18: safe_normalize_with_bool_success,
    }
    for _, token_row in tokens_and_decimals.iterrows():
        token = token_row["token_address"]
        normalizer = decimal_normalizer_map[token_row["decimals"]]
        call = Call(
            token,
            ["balanceOf(address)(uint256)", rebalance_event_row["solver_address"]],
            [(token, normalizer)],
        )
        balance_of_calls.append(call)

    balance_of_df = get_raw_state_by_blocks(
        balance_of_calls,
        [rebalance_event_row["block"] - 1, rebalance_event_row["block"]],
        chain=autopool.chain,
    )
    return balance_of_df


sus_row = df[df["block"] == 22335534].iloc[0]
sus_row.T

tx_hash                            0x6ab3f09e6a2ec4ef83c2dbd851f3c92a10f8fd85a611...
autopool_vault_address                    0xa7569A44f348d3D70d8ad5889e50F78E33d80D35
chain_id                                                                           1
rebalance_file_path                rebalance_plan_1745457253_0xa7569A44f348d3D70d...
destination_out                           0xa7569A44f348d3D70d8ad5889e50F78E33d80D35
destination_in                            0x65efCF2cce562DCBf07e805eEbeDeF21Dbd8Ea3D
quantity_out                                                                150000.0
quantity_in                                                            148393.790366
safe_value_out                                                              150000.0
safe_value_in                                                          151118.151963
spot_value_in                                                          151112.216211
spot_value_out                                                   

In [140]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor


# ---------- Function to fetch balances for a single row ----------
def _fetch_some_non_zero_changes(rebalance_event_row: dict) -> pd.DataFrame:
    tokens_and_decimals = destination_info_df[["token_address", "decimals"]].drop_duplicates()
    balance_of_calls = []
    decimal_normalizer_map = {
        6: safe_normalize_6_with_bool_success,
        18: safe_normalize_with_bool_success,
    }
    for _, token_row in tokens_and_decimals.iterrows():
        token = token_row["token_address"]
        normalizer = decimal_normalizer_map[token_row["decimals"]]
        call = Call(
            token,
            ["balanceOf(address)(uint256)", rebalance_event_row["solver_address"]],
            [(token, normalizer)],
        )
        balance_of_calls.append(call)

    balance_of_df = get_raw_state_by_blocks(
        balance_of_calls,
        [rebalance_event_row["block"] - 1, rebalance_event_row["block"]],
        chain=autopool.chain,
    )
    return balance_of_df


# ---------- Apply over each row and stitch together without extra indexes ----------
def fetch_and_concat_all(df: pd.DataFrame) -> pd.DataFrame:
    rows = df.to_dict("records")
    with ThreadPoolExecutor() as executor:
        dfs = list(executor.map(_fetch_some_non_zero_changes, rows))
    return pd.concat(dfs).sort_index()


# ---------- Usage example ----------
large_df = fetch_and_concat_all(df)
df

,tx_hash,autopool_vault_address,chain_id,rebalance_file_path,destination_out,destination_in,quantity_out,quantity_in,safe_value_out,safe_value_in,spot_value_in,spot_value_out,swap_offset_period,block,destination_in_symbol,destination_out_symbol,destination_in_tokens,destination_out_tokens,tokens_move_name,spot_swap_cost,spot_slippage_bps,safe_swap_cost,safe_slippage_bps,file_name,datetime_generated,solver_address,rebalance_type,token_out,token_in,move_name,amount_out,amount_out_safe_value,min_amount_in,min_amount_in_safe_value,amount_out_spot_value,out_dest_apr,min_amount_in_spot_value,in_dest_apr,in_dest_adj_apr,apr_delta,num_candidate_destinations,candidate_destinations_rank,projected_swap_cost,projected_net_gain,projected_gross_gain,projected_slippage,spot_value_difference_in_solver,real_spot_swap_cost,real_spot_slippage
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-04-03 19:30:11+00:00,0x67618958fa10c91e18218c3c3f7eb57509e70445e898...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743708477_0xa7569A44f348d3D70d...,0xb6E89d23d1E33537D383622B1955EbDa135c0b5A,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,85.899838,100.008658,100.118752,100.008658,100.008658,100.067556,None,22190481,autoUSD,sUSDe,"('USDC',)","('sUSDe',)","('sUSDe',) -> ('USDC',)",0.058898,5.885779,0.110094,10.996327,rebalance_plan_1743708477_0xa7569A44f348d3D70d...,2025-04-03 19:27:57+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,ToIdle,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,sUSDe -> autoUSD,85.899838,1.001188e-10,99.888696,9.992867e-11,100.067556,0.061408,99.928668,0.061408,0.000000,-0.061408,0.0,NaN,1.900830e-13,1.900830e-13,0.000000,0.189858,0.015010,0.043887,4.385779
2025-04-03 23:46:35+00:00,0x5513250273bfdbf939553c9ed98ce48b8479bc739634...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743723845_0xa7569A44f348d3D70d...,0x6F628dcCD275feA4277722D177265741031E09d7,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,98.326351,99.954654,99.992786,99.954654,99.954654,99.980200,None,22191759,autoUSD,crvUSDUSDT-f,"('USDC',)","('crvUSD', 'USDT')","('crvUSD', 'USDT') -> ('USDC',)",0.025546,2.555152,0.038132,3.813499,rebalance_plan_1743723845_0xa7569A44f348d3D70d...,2025-04-03 23:44:05+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,ToIdle,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,crvUSDUSDT-f -> autoUSD,98.326351,9.999279e-11,99.914672,9.995465e-11,99.980200,0.091285,99.954654,0.091285,0.000000,-0.091285,0.0,NaN,3.813300e-14,3.813300e-14,0.000000,0.038136,0.014997,0.010550,1.055162
2025-04-04 15:57:35+00:00,0x14425c16bdc198d43d738991130a7fea735c85d7c883...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743782101_0xa7569A44f348d3D70d...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,25503.391475,25348.990383,25503.391475,25505.925983,25502.655963,25503.391475,None,22196597,crv2pool,autoUSD,"('USDT', 'USDC')","('USDC',)","('USDC',) -> ('USDT', 'USDC')",0.735512,0.288398,-2.534508,-0.993792,rebalance_plan_1743782101_0xa7569A44f348d3D70d...,2025-04-04 15:55:01+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,autoUSD -> crv2pool,25503.391475,2.550339e-08,25335.042437,2.550210e-08,25503.391475,0.000000,25498.822994,0.000000,0.090720,0.090720,12.0,0.0,1.287086e-12,7.140939e+01,71.409385,0.005047,-0.008092,0.743604,0.291571
2025-04-04 17:11:35+00:00,0x5c3a705003f40a1e38309a79c873f54f147043dbcdd4...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743786510_0xa7569A44f348d3D70d...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x7583b1589aDD33320366A48A92794D77763FAE9e,19127.543600,18806.816895,19127.543600,19124.181930,19122.451703,19127.543600,None,22196965,crvUSDUSDT-f,autoUSD,"('USDT', 'crvUSD')","('USDC',)","('USDC',) -> ('USDT', 'crvUSD')",5.091897,2.662076,3.361670,1.757502,rebalance_pla

In [172]:
px.line(large_df)

In [ ]:
0.8 * 6

4.800000000000001

In [162]:
df[df["spot_value_difference_in_solver"] > 0]["spot_value_difference_in_solver"].sum()

np.float64(2435.3274623829793)

In [164]:
df["spot_value_out"].sum()

np.float64(59639733.34432648)

In [163]:
df[df["spot_value_difference_in_solver"] < 0]["spot_value_difference_in_solver"].sum()

np.float64(-5456.735070943838)

In [158]:
large_df.columns

Index(['0x83F20F44975D03b1b09e64809B757c47f942BEeA',
       '0x0655977FEb2f289A4aB78af67BAB0d17aAb84367',
       '0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E',
       '0x853d955aCEf822Db058eb8505911ED77F175b99e',
       '0x9D39A5DE30e57443BfF2A8307A4256c8797A3497',
       '0xa3931d71877C0E7a3148CB7Eb4463524FEc27fbD',
       '0xD4fa2D31b7968E448877f69A96DE69f5de8cD23E',
       '0xdAC17F958D2ee523a2206206994597C13D831ec7',
       '0x7Bc3485026Ac48b6cf9BaF0A377477Fff5703Af8',
       '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
       '0xA663B02CF0a4b149d2aD41910CB81e23e1c41c32',
       '0xC71Ea051a5F82c67ADcF634c36FFE6334793D24C',
       '0x4c9EDD5852cd905f086C759E8383e09bff1E68B3',
       '0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f',
       '0xCAcd6fd266aF91b8AeD52aCCc382b4e165586E29',
       '0xcf62F905562626CfcDD2261162a51fd02Fc9c5b6'],
      dtype='object')

In [155]:
df.columns

Index(['tx_hash', 'autopool_vault_address', 'chain_id', 'rebalance_file_path',
       'destination_out', 'destination_in', 'quantity_out', 'quantity_in',
       'safe_value_out', 'safe_value_in', 'spot_value_in', 'spot_value_out',
       'swap_offset_period', 'block', 'destination_in_symbol',
       'destination_out_symbol', 'destination_in_tokens',
       'destination_out_tokens', 'tokens_move_name', 'spot_swap_cost',
       'spot_slippage_bps', 'safe_swap_cost', 'safe_slippage_bps', 'file_name',
       'datetime_generated', 'solver_address', 'rebalance_type', 'token_out',
       'token_in', 'move_name', 'amount_out', 'amount_out_safe_value',
       'min_amount_in', 'min_amount_in_safe_value', 'amount_out_spot_value',
       'out_dest_apr', 'min_amount_in_spot_value', 'in_dest_apr',
       'in_dest_adj_apr', 'apr_delta', 'num_candidate_destinations',
       'candidate_destinations_rank', 'projected_swap_cost',
       'projected_net_gain', 'projected_gross_gain', 'projected_slippage',


In [ ]:
sus_row = df[df["block"] == 22335534].iloc
sus_row

tx_hash                            0x6ab3f09e6a2ec4ef83c2dbd851f3c92a10f8fd85a611...
autopool_vault_address                    0xa7569A44f348d3D70d8ad5889e50F78E33d80D35
chain_id                                                                           1
rebalance_file_path                rebalance_plan_1745457253_0xa7569A44f348d3D70d...
destination_out                           0xa7569A44f348d3D70d8ad5889e50F78E33d80D35
destination_in                            0x65efCF2cce562DCBf07e805eEbeDeF21Dbd8Ea3D
quantity_out                                                                150000.0
quantity_in                                                            148393.790366
safe_value_out                                                              150000.0
safe_value_in                                                          151118.151963
spot_value_in                                                          151112.216211
spot_value_out                                                   

In [134]:
_fetch_some_non_zero_changes(sus_row)

{'0x83F20F44975D03b1b09e64809B757c47f942BEeA': 0.0,
 '0x0655977FEb2f289A4aB78af67BAB0d17aAb84367': 0.0,
 '0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E': 0.0,
 '0x853d955aCEf822Db058eb8505911ED77F175b99e': 0.0,
 '0x9D39A5DE30e57443BfF2A8307A4256c8797A3497': 0.0,
 '0xa3931d71877C0E7a3148CB7Eb4463524FEc27fbD': 0.0,
 '0xD4fa2D31b7968E448877f69A96DE69f5de8cD23E': 0.0,
 '0xdAC17F958D2ee523a2206206994597C13D831ec7': 0.0,
 '0x7Bc3485026Ac48b6cf9BaF0A377477Fff5703Af8': 0.0,
 '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48': -1144.43124,
 '0xA663B02CF0a4b149d2aD41910CB81e23e1c41c32': 0.0,
 '0xC71Ea051a5F82c67ADcF634c36FFE6334793D24C': 0.0,
 '0x4c9EDD5852cd905f086C759E8383e09bff1E68B3': 0.0,
 '0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f': 0.0,
 '0xCAcd6fd266aF91b8AeD52aCCc382b4e165586E29': 0.0,
 '0xcf62F905562626CfcDD2261162a51fd02Fc9c5b6': 0.0}

In [202]:
user = "0x9A83da4AfF68643C5215AC6383cC026d90800262"

total_value_in = 16_957 + 100_770

shares = 16_938 + 100_640
share_of_year = 10 / 365
total_value_in

shares, total_value_in

(117578, 117727)

In [209]:
current_share_value = 117789854664 / 1e6

In [214]:
((current_share_value - total_value_in) / total_value_in) / share_of_year

0.01948741780560066

In [211]:
(current_share_value - total_value_in) / share_of_year

2294.1952359999486

In [204]:
int(117578 * 1e18)

117578000000000001572864

In [128]:
df

,tx_hash,autopool_vault_address,chain_id,rebalance_file_path,destination_out,destination_in,quantity_out,quantity_in,safe_value_out,safe_value_in,spot_value_in,spot_value_out,swap_offset_period,block,destination_in_symbol,destination_out_symbol,destination_in_tokens,destination_out_tokens,tokens_move_name,spot_swap_cost,spot_slippage_bps,safe_swap_cost,safe_slippage_bps,file_name,datetime_generated,solver_address,rebalance_type,token_out,token_in,move_name,amount_out,amount_out_safe_value,min_amount_in,min_amount_in_safe_value,amount_out_spot_value,out_dest_apr,min_amount_in_spot_value,in_dest_apr,in_dest_adj_apr,apr_delta,num_candidate_destinations,candidate_destinations_rank,projected_swap_cost,projected_net_gain,projected_gross_gain,projected_slippage,spot_value_difference_in_solver,real_spot_swap_cost,real_spot_slippage
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-04-03 19:30:11+00:00,0x67618958fa10c91e18218c3c3f7eb57509e70445e898...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743708477_0xa7569A44f348d3D70d...,0xb6E89d23d1E33537D383622B1955EbDa135c0b5A,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,85.899838,100.008658,100.118752,100.008658,100.008658,100.067556,None,22190481,autoUSD,sUSDe,"('USDC',)","('sUSDe',)","('sUSDe',) -> ('USDC',)",0.058898,5.885779,0.110094,10.996327,rebalance_plan_1743708477_0xa7569A44f348d3D70d...,2025-04-03 19:27:57+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,ToIdle,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,sUSDe -> autoUSD,85.899838,1.001188e-10,99.888696,9.992867e-11,100.067556,0.061408,99.928668,0.061408,0.000000,-0.061408,0.0,NaN,1.900830e-13,1.900830e-13,0.000000,0.189858,0.015010,0.043887,4.385779
2025-04-03 23:46:35+00:00,0x5513250273bfdbf939553c9ed98ce48b8479bc739634...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743723845_0xa7569A44f348d3D70d...,0x6F628dcCD275feA4277722D177265741031E09d7,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,98.326351,99.954654,99.992786,99.954654,99.954654,99.980200,None,22191759,autoUSD,crvUSDUSDT-f,"('USDC',)","('crvUSD', 'USDT')","('crvUSD', 'USDT') -> ('USDC',)",0.025546,2.555152,0.038132,3.813499,rebalance_plan_1743723845_0xa7569A44f348d3D70d...,2025-04-03 23:44:05+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,ToIdle,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,crvUSDUSDT-f -> autoUSD,98.326351,9.999279e-11,99.914672,9.995465e-11,99.980200,0.091285,99.954654,0.091285,0.000000,-0.091285,0.0,NaN,3.813300e-14,3.813300e-14,0.000000,0.038136,0.014997,0.010550,1.055162
2025-04-04 15:57:35+00:00,0x14425c16bdc198d43d738991130a7fea735c85d7c883...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743782101_0xa7569A44f348d3D70d...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,25503.391475,25348.990383,25503.391475,25505.925983,25502.655963,25503.391475,None,22196597,crv2pool,autoUSD,"('USDT', 'USDC')","('USDC',)","('USDC',) -> ('USDT', 'USDC')",0.735512,0.288398,-2.534508,-0.993792,rebalance_plan_1743782101_0xa7569A44f348d3D70d...,2025-04-04 15:55:01+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,autoUSD -> crv2pool,25503.391475,2.550339e-08,25335.042437,2.550210e-08,25503.391475,0.000000,25498.822994,0.000000,0.090720,0.090720,12.0,0.0,1.287086e-12,7.140939e+01,71.409385,0.005047,-0.008092,0.743604,0.291571
2025-04-04 17:11:35+00:00,0x5c3a705003f40a1e38309a79c873f54f147043dbcdd4...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743786510_0xa7569A44f348d3D70d...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x7583b1589aDD33320366A48A92794D77763FAE9e,19127.543600,18806.816895,19127.543600,19124.181930,19122.451703,19127.543600,None,22196965,crvUSDUSDT-f,autoUSD,"('USDT', 'crvUSD')","('USDC',)","('USDC',) -> ('USDT', 'crvUSD')",5.091897,2.662076,3.361670,1.757502,rebalance_pla

In [126]:
df

,tx_hash,autopool_vault_address,chain_id,rebalance_file_path,destination_out,destination_in,quantity_out,quantity_in,safe_value_out,safe_value_in,spot_value_in,spot_value_out,swap_offset_period,block,destination_in_symbol,destination_out_symbol,destination_in_tokens,destination_out_tokens,tokens_move_name,spot_swap_cost,spot_slippage_bps,safe_swap_cost,safe_slippage_bps,file_name,datetime_generated,solver_address,rebalance_type,token_out,token_in,move_name,amount_out,amount_out_safe_value,min_amount_in,min_amount_in_safe_value,amount_out_spot_value,out_dest_apr,min_amount_in_spot_value,in_dest_apr,in_dest_adj_apr,apr_delta,num_candidate_destinations,candidate_destinations_rank,projected_swap_cost,projected_net_gain,projected_gross_gain,projected_slippage,spot_value_difference_in_solver,real_spot_swap_cost,real_spot_slippage
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-04-03 19:30:11+00:00,0x67618958fa10c91e18218c3c3f7eb57509e70445e898...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743708477_0xa7569A44f348d3D70d...,0xb6E89d23d1E33537D383622B1955EbDa135c0b5A,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,85.899838,100.008658,100.118752,100.008658,100.008658,100.067556,None,22190481,autoUSD,sUSDe,"('USDC',)","('sUSDe',)","('sUSDe',) -> ('USDC',)",0.058898,5.885779,0.110094,10.996327,rebalance_plan_1743708477_0xa7569A44f348d3D70d...,2025-04-03 19:27:57+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,ToIdle,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,sUSDe -> autoUSD,85.899838,1.001188e-10,99.888696,9.992867e-11,100.067556,0.061408,99.928668,0.061408,0.000000,-0.061408,0.0,NaN,1.900830e-13,1.900830e-13,0.000000,0.189858,0.015010,0.043887,4.385779
2025-04-03 23:46:35+00:00,0x5513250273bfdbf939553c9ed98ce48b8479bc739634...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743723845_0xa7569A44f348d3D70d...,0x6F628dcCD275feA4277722D177265741031E09d7,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,98.326351,99.954654,99.992786,99.954654,99.954654,99.980200,None,22191759,autoUSD,crvUSDUSDT-f,"('USDC',)","('crvUSD', 'USDT')","('crvUSD', 'USDT') -> ('USDC',)",0.025546,2.555152,0.038132,3.813499,rebalance_plan_1743723845_0xa7569A44f348d3D70d...,2025-04-03 23:44:05+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,ToIdle,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,crvUSDUSDT-f -> autoUSD,98.326351,9.999279e-11,99.914672,9.995465e-11,99.980200,0.091285,99.954654,0.091285,0.000000,-0.091285,0.0,NaN,3.813300e-14,3.813300e-14,0.000000,0.038136,0.014997,0.010550,1.055162
2025-04-04 15:57:35+00:00,0x14425c16bdc198d43d738991130a7fea735c85d7c883...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743782101_0xa7569A44f348d3D70d...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,25503.391475,25348.990383,25503.391475,25505.925983,25502.655963,25503.391475,None,22196597,crv2pool,autoUSD,"('USDT', 'USDC')","('USDC',)","('USDC',) -> ('USDT', 'USDC')",0.735512,0.288398,-2.534508,-0.993792,rebalance_plan_1743782101_0xa7569A44f348d3D70d...,2025-04-04 15:55:01+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,autoUSD -> crv2pool,25503.391475,2.550339e-08,25335.042437,2.550210e-08,25503.391475,0.000000,25498.822994,0.000000,0.090720,0.090720,12.0,0.0,1.287086e-12,7.140939e+01,71.409385,0.005047,-0.008092,0.743604,0.291571
2025-04-04 17:11:35+00:00,0x5c3a705003f40a1e38309a79c873f54f147043dbcdd4...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743786510_0xa7569A44f348d3D70d...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x7583b1589aDD33320366A48A92794D77763FAE9e,19127.543600,18806.816895,19127.543600,19124.181930,19122.451703,19127.543600,None,22196965,crvUSDUSDT-f,autoUSD,"('USDT', 'crvUSD')","('USDC',)","('USDC',) -> ('USDT', 'crvUSD')",5.091897,2.662076,3.361670,1.757502,rebalance_pla

In [193]:
10000 * (13500 / 60_000_000)

2.25

In [167]:
df

,tx_hash,autopool_vault_address,chain_id,rebalance_file_path,destination_out,destination_in,quantity_out,quantity_in,safe_value_out,safe_value_in,spot_value_in,spot_value_out,swap_offset_period,block,destination_in_symbol,destination_out_symbol,destination_in_tokens,destination_out_tokens,tokens_move_name,spot_swap_cost,spot_slippage_bps,safe_swap_cost,safe_slippage_bps,file_name,datetime_generated,solver_address,rebalance_type,token_out,token_in,move_name,amount_out,amount_out_safe_value,min_amount_in,min_amount_in_safe_value,amount_out_spot_value,out_dest_apr,min_amount_in_spot_value,in_dest_apr,in_dest_adj_apr,apr_delta,num_candidate_destinations,candidate_destinations_rank,projected_swap_cost,projected_net_gain,projected_gross_gain,projected_slippage,spot_value_difference_in_solver,real_spot_swap_cost,real_spot_slippage
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-04-03 19:30:11+00:00,0x67618958fa10c91e18218c3c3f7eb57509e70445e898...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743708477_0xa7569A44f348d3D70d...,0xb6E89d23d1E33537D383622B1955EbDa135c0b5A,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,85.899838,100.008658,100.118752,100.008658,100.008658,100.067556,None,22190481,autoUSD,sUSDe,"('USDC',)","('sUSDe',)","('sUSDe',) -> ('USDC',)",0.058898,5.885779,0.110094,10.996327,rebalance_plan_1743708477_0xa7569A44f348d3D70d...,2025-04-03 19:27:57+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,ToIdle,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,sUSDe -> autoUSD,85.899838,1.001188e-10,99.888696,9.992867e-11,100.067556,0.061408,99.928668,0.061408,0.000000,-0.061408,0.0,NaN,1.900830e-13,1.900830e-13,0.000000,0.189858,0.015010,0.043887,4.385779
2025-04-03 23:46:35+00:00,0x5513250273bfdbf939553c9ed98ce48b8479bc739634...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743723845_0xa7569A44f348d3D70d...,0x6F628dcCD275feA4277722D177265741031E09d7,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,98.326351,99.954654,99.992786,99.954654,99.954654,99.980200,None,22191759,autoUSD,crvUSDUSDT-f,"('USDC',)","('crvUSD', 'USDT')","('crvUSD', 'USDT') -> ('USDC',)",0.025546,2.555152,0.038132,3.813499,rebalance_plan_1743723845_0xa7569A44f348d3D70d...,2025-04-03 23:44:05+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,ToIdle,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,crvUSDUSDT-f -> autoUSD,98.326351,9.999279e-11,99.914672,9.995465e-11,99.980200,0.091285,99.954654,0.091285,0.000000,-0.091285,0.0,NaN,3.813300e-14,3.813300e-14,0.000000,0.038136,0.014997,0.010550,1.055162
2025-04-04 15:57:35+00:00,0x14425c16bdc198d43d738991130a7fea735c85d7c883...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743782101_0xa7569A44f348d3D70d...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,25503.391475,25348.990383,25503.391475,25505.925983,25502.655963,25503.391475,None,22196597,crv2pool,autoUSD,"('USDT', 'USDC')","('USDC',)","('USDC',) -> ('USDT', 'USDC')",0.735512,0.288398,-2.534508,-0.993792,rebalance_plan_1743782101_0xa7569A44f348d3D70d...,2025-04-04 15:55:01+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,autoUSD -> crv2pool,25503.391475,2.550339e-08,25335.042437,2.550210e-08,25503.391475,0.000000,25498.822994,0.000000,0.090720,0.090720,12.0,0.0,1.287086e-12,7.140939e+01,71.409385,0.005047,-0.008092,0.743604,0.291571
2025-04-04 17:11:35+00:00,0x5c3a705003f40a1e38309a79c873f54f147043dbcdd4...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743786510_0xa7569A44f348d3D70d...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x7583b1589aDD33320366A48A92794D77763FAE9e,19127.543600,18806.816895,19127.543600,19124.181930,19122.451703,19127.543600,None,22196965,crvUSDUSDT-f,autoUSD,"('USDT', 'crvUSD')","('USDC',)","('USDC',) -> ('USDT', 'crvUSD')",5.091897,2.662076,3.361670,1.757502,rebalance_pla

In [156]:
df["real_spot_swap_cost"].sum()

np.float64(13583.995911452654)

In [ ]:
df = df.set_index("datetime")

In [117]:
df.groupby("move_name")["real_spot_swap_cost"].sum().round(3).sort_values()

move_name
autoUSD -> fUSDC            -1742.848
crvUSDUSDT-f -> fUSDC        -181.979
crv2pool -> crvU-sUSDe        -58.858
crvUSDe -> frxsUSDS           -53.865
fUSDT -> frxsUSDS             -39.083
fUSDT -> crvU-sUSDe           -37.947
crv2pool -> crvUSDUSDC-f      -32.587
autoUSD -> gtUSDCcore         -25.470
sUSDe/USDC -> fUSDC           -16.822
crv2pool -> fUSDC             -10.066
autoUSD -> gtUSDC              -5.802
gtUSDC -> SkySaver             -4.843
sUSDe/USDC -> frxsUSDS         -4.669
crvUSDUSDT-f -> autoUSD         0.011
gtUSDCcore -> fUSDC             0.022
sUSDe -> autoUSD                0.044
gtUSDC -> fUSDC                 0.050
fUSDC -> gtUSDCcore             0.057
steakUSDC -> fUSDC              0.058
steakUSDC -> crvUSDUSDC-f       0.165
autoUSD -> waEthUSDT            1.584
crvUSDUSDT-f -> crvUSDe         1.632
autoUSD -> steakUSDC            6.971
waEthUSDT -> SkySaver           7.286
fUSDT -> SkySaver              10.625
fUSDC -> crvUSDUSDC-f          12.621
fr

In [173]:
import plotly.express as px
import plotly.io as pio

pio.templates.default = None

df["adjusted_spot_value_out"] = df["spot_value_out"]
by_move_name = df.groupby("move_name")[["spot_value_out", "real_spot_swap_cost"]].sum()
by_move_name["all_time_spot_slippage"] = 10_000 * by_move_name["real_spot_swap_cost"] / by_move_name["spot_value_out"]

px.bar(by_move_name["all_time_spot_slippage"], orientation="v").show()

px.scatter(by_move_name, x="spot_value_out", y="all_time_spot_slippage", color=by_move_name.index)

In [ ]:
# look at the different dexes,

In [185]:
by_move_name.sort_values("all_time_spot_slippage")["all_time_spot_slippage"].tail(15)

move_name
autoUSD -> frxsUSDS         2.970365
autoUSD -> SkySaver         3.226493
autoUSD -> crvUSDUSDC-f     3.237775
autoUSD -> fUSDT            3.808861
frxsUSDS -> SkySaver        4.199825
fUSDT -> GHOcrvUSD          4.241787
autoUSD -> GHOcrvUSD        4.332276
sUSDe -> autoUSD            4.385779
crvUSDe -> crvUSDUSDC-f     4.567400
autoUSD -> crvUSDUSDT-f     5.582303
autoUSD -> crv2pool         5.653568
fUSDT -> sUSDe/USDC         5.720592
crv2pool -> frxsUSDS        5.724913
autoUSD -> crvUSDe          5.860061
crv2pool -> waEthUSDT      19.784558
Name: all_time_spot_slippage, dtype: float64

In [186]:
by_move_name[by_move_name["all_time_spot_slippage"] > 4]

,spot_value_out,real_spot_swap_cost,all_time_spot_slippage
move_name,,,
autoUSD -> GHOcrvUSD,1.967810e+06,852.509535,4.332276
autoUSD -> crv2pool,6.615255e+06,3739.979119,5.653568
autoUSD -> crvUSDUSDT-f,4.562905e+06,2547.151722,5.582303
autoUSD -> crvUSDe,1.860202e+06,1090.089463,5.860061
crv2pool -> frxsUSDS,2.000000e+05,114.498252,5.724913
crv2pool -> waEthUSDT,2.000000e+05,395.691153,19.784558
crvUSDe -> crvUSDUSDC-f,1.000000e+05,45.674002,4.567400
fUSDT -> GHOcrvUSD,3.000000e+05,127.253619,4.241787
fUSDT -> sUSDe/USDC,1.500000e+05,85.808879,5.720592


In [187]:
by_move_name[by_move_name["all_time_spot_slippage"] > 5]

,spot_value_out,real_spot_swap_cost,all_time_spot_slippage
move_name,,,
autoUSD -> crv2pool,6.615255e+06,3739.979119,5.653568
autoUSD -> crvUSDUSDT-f,4.562905e+06,2547.151722,5.582303
autoUSD -> crvUSDe,1.860202e+06,1090.089463,5.860061
crv2pool -> frxsUSDS,2.000000e+05,114.498252,5.724913
crv2pool -> waEthUSDT,2.000000e+05,395.691153,19.784558
fUSDT -> sUSDe/USDC,1.500000e+05,85.808879,5.720592


In [190]:
10_000 * by_move_name["real_spot_swap_cost"].sum() / by_move_name["spot_value_out"].sum()

np.float64(2.2776754941250745)

In [182]:
by_move_name.sort_values("all_time_spot_slippage")["all_time_spot_slippage"].head()

move_name
crvUSDUSDT-f -> fUSDC      -4.549465
fUSDT -> crvU-sUSDe        -2.529804
crv2pool -> crvU-sUSDe     -2.354328
crv2pool -> crvUSDUSDC-f   -2.172473
sUSDe/USDC -> fUSDC        -2.162817
Name: all_time_spot_slippage, dtype: float64

Index(['tx_hash', 'autopool_vault_address', 'chain_id', 'rebalance_file_path',
       'destination_out', 'destination_in', 'quantity_out', 'quantity_in',
       'safe_value_out', 'safe_value_in', 'spot_value_in', 'spot_value_out',
       'swap_offset_period', 'block', 'destination_in_symbol',
       'destination_out_symbol', 'destination_in_tokens',
       'destination_out_tokens', 'tokens_move_name', 'spot_swap_cost',
       'spot_slippage_bps', 'safe_swap_cost', 'safe_slippage_bps', 'file_name',
       'datetime_generated', 'solver_address', 'rebalance_type', 'token_out',
       'token_in', 'move_name', 'amount_out', 'amount_out_safe_value',
       'min_amount_in', 'min_amount_in_safe_value', 'amount_out_spot_value',
       'out_dest_apr', 'min_amount_in_spot_value', 'in_dest_apr',
       'in_dest_adj_apr', 'apr_delta', 'num_candidate_destinations',
       'candidate_destinations_rank', 'projected_swap_cost',
       'projected_net_gain', 'projected_gross_gain', 'projected_slippage',


In [177]:
df

,tx_hash,autopool_vault_address,chain_id,rebalance_file_path,destination_out,destination_in,quantity_out,quantity_in,safe_value_out,safe_value_in,spot_value_in,spot_value_out,swap_offset_period,block,destination_in_symbol,destination_out_symbol,destination_in_tokens,destination_out_tokens,tokens_move_name,spot_swap_cost,spot_slippage_bps,safe_swap_cost,safe_slippage_bps,file_name,datetime_generated,solver_address,rebalance_type,token_out,token_in,move_name,amount_out,amount_out_safe_value,min_amount_in,min_amount_in_safe_value,amount_out_spot_value,out_dest_apr,min_amount_in_spot_value,in_dest_apr,in_dest_adj_apr,apr_delta,num_candidate_destinations,candidate_destinations_rank,projected_swap_cost,projected_net_gain,projected_gross_gain,projected_slippage,spot_value_difference_in_solver,real_spot_swap_cost,real_spot_slippage,adjusted_spot_value_out
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-04-03 19:30:11+00:00,0x67618958fa10c91e18218c3c3f7eb57509e70445e898...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743708477_0xa7569A44f348d3D70d...,0xb6E89d23d1E33537D383622B1955EbDa135c0b5A,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,85.899838,100.008658,100.118752,100.008658,100.008658,100.067556,None,22190481,autoUSD,sUSDe,"('USDC',)","('sUSDe',)","('sUSDe',) -> ('USDC',)",0.058898,5.885779,0.110094,10.996327,rebalance_plan_1743708477_0xa7569A44f348d3D70d...,2025-04-03 19:27:57+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,ToIdle,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,sUSDe -> autoUSD,85.899838,1.001188e-10,99.888696,9.992867e-11,100.067556,0.061408,99.928668,0.061408,0.000000,-0.061408,0.0,NaN,1.900830e-13,1.900830e-13,0.000000,0.189858,0.015010,0.043887,4.385779,100.067556
2025-04-03 23:46:35+00:00,0x5513250273bfdbf939553c9ed98ce48b8479bc739634...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743723845_0xa7569A44f348d3D70d...,0x6F628dcCD275feA4277722D177265741031E09d7,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,98.326351,99.954654,99.992786,99.954654,99.954654,99.980200,None,22191759,autoUSD,crvUSDUSDT-f,"('USDC',)","('crvUSD', 'USDT')","('crvUSD', 'USDT') -> ('USDC',)",0.025546,2.555152,0.038132,3.813499,rebalance_plan_1743723845_0xa7569A44f348d3D70d...,2025-04-03 23:44:05+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,ToIdle,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,crvUSDUSDT-f -> autoUSD,98.326351,9.999279e-11,99.914672,9.995465e-11,99.980200,0.091285,99.954654,0.091285,0.000000,-0.091285,0.0,NaN,3.813300e-14,3.813300e-14,0.000000,0.038136,0.014997,0.010550,1.055162,99.980200
2025-04-04 15:57:35+00:00,0x14425c16bdc198d43d738991130a7fea735c85d7c883...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743782101_0xa7569A44f348d3D70d...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,25503.391475,25348.990383,25503.391475,25505.925983,25502.655963,25503.391475,None,22196597,crv2pool,autoUSD,"('USDT', 'USDC')","('USDC',)","('USDC',) -> ('USDT', 'USDC')",0.735512,0.288398,-2.534508,-0.993792,rebalance_plan_1743782101_0xa7569A44f348d3D70d...,2025-04-04 15:55:01+00:00,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,autoUSD -> crv2pool,25503.391475,2.550339e-08,25335.042437,2.550210e-08,25503.391475,0.000000,25498.822994,0.000000,0.090720,0.090720,12.0,0.0,1.287086e-12,7.140939e+01,71.409385,0.005047,-0.008092,0.743604,0.291571,25503.391475
2025-04-04 17:11:35+00:00,0x5c3a705003f40a1e38309a79c873f54f147043dbcdd4...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743786510_0xa7569A44f348d3D70d...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x7583b1589aDD33320366A48A92794D77763FAE9e,19127.543600,18806.816895,19127.543600,19124.181930,19122.451703,19127.543600,None,22196965,crvUSDUSDT-f,autoUSD,"('USDT', 'crvUSD')","('USDC',)","('USDC',) -> ('USDT', '

In [ ]:
0xA7569A44F348D3D70D8AD5889E50F78E33D80D35

0xBB2D2DD491204A86EC10A1A6972F940B34FE060E

In [175]:
destination_info_df

,destination_vault_address,autopool_vault_address,token_address,pool,decimals,name
0,0x04ee6c666a01018a070925040ea2aF6554d051e4,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x83F20F44975D03b1b09e64809B757c47f942BEeA,0x167478921b907422F8E88B43C4Af2B8BEa278d3A,18,Savings Dai
1,0x53C8C211350b635269e02B2C2f1077b850C657dB,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x0655977FEb2f289A4aB78af67BAB0d17aAb84367,0xd29f8980852c2c76fC3f6E96a7Aa06E0BedCC1B1,18,Savings crvUSD
2,0x65efCF2cce562DCBf07e805eEbeDeF21Dbd8Ea3D,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E,0x4DEcE678ceceb27446b35C672dC7d61F30bAD69E,18,Curve.Fi USD Stablecoin
3,0x9906eB64BA32Fb4fD3e5541ecA95e57610084d02,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x853d955aCEf822Db058eb8505911ED77F175b99e,0x0CD6f267b2086bea681E922E19D40512511BE538,18,Frax
4,0x3Ee3F7A0B244004B0678EBa98bA091eC101513Ee,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x0655977FEb2f289A4aB78af67BAB0d17aAb84367,0xfD1627E3f3469C8392C8c3A261D8F0677586e5e1,18,Savings crvUSD
...,...,...,...,...,...,...
77,0x769C6bEA7db25F4c0a52f0dB0960c42F708DBF41,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xD4fa2D31b7968E448877f69A96DE69f5de8cD23E,6,USD Coin
78,0x9Fe20B75709403723610Fd2995D0f4A9e19C3514,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,6,USD Coin
79,0xEeEE628a00B0EDa99D1bc78a72Fa2F2Df4AdE2F4,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E,0x0CD6f267b2086bea681E922E19D40512511BE538,18,Curve.Fi USD Stablecoin
80,0x9b18801C9E95f45AD0aDD54ef2Eb4690c44d170d,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xcf62F905562626CfcDD2261162a51fd02Fc9c5b6,0xcf62F905562626CfcDD2261162a51fd02Fc9c5b6,18,Staked Frax USD


In [174]:
px.scatter(df, x="spot_value_out", y="all_time_spot_slippage", color=by_move_name.index)

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['tx_hash', 'autopool_vault_address', 'chain_id', 'rebalance_file_path', 'destination_out', 'destination_in', 'quantity_out', 'quantity_in', 'safe_value_out', 'safe_value_in', 'spot_value_in', 'spot_value_out', 'swap_offset_period', 'block', 'destination_in_symbol', 'destination_out_symbol', 'destination_in_tokens', 'destination_out_tokens', 'tokens_move_name', 'spot_swap_cost', 'spot_slippage_bps', 'safe_swap_cost', 'safe_slippage_bps', 'file_name', 'datetime_generated', 'solver_address', 'rebalance_type', 'token_out', 'token_in', 'move_name', 'amount_out', 'amount_out_safe_value', 'min_amount_in', 'min_amount_in_safe_value', 'amount_out_spot_value', 'out_dest_apr', 'min_amount_in_spot_value', 'in_dest_apr', 'in_dest_adj_apr', 'apr_delta', 'num_candidate_destinations', 'candidate_destinations_rank', 'projected_swap_cost', 'projected_net_gain', 'projected_gross_gain', 'projected_slippage', 'spot_value

In [171]:
px.scatter(by_move_name, x="real_spot_swap_cost", y="all_time_spot_slippage", color=by_move_name.index)

In [ ]:
# see what is up with the high waETHUSDT pool borrowing
# figure out why whe

In [170]:
by_move_name["all_time_spot_slippage"]

move_name
autoUSD -> GHOcrvUSD          4.332276
autoUSD -> SkySaver           3.226493
autoUSD -> crv2pool           5.653568
autoUSD -> crvU-sUSDe         1.781764
autoUSD -> crvUSDUSDC-f       3.237775
autoUSD -> crvUSDUSDT-f       5.582303
autoUSD -> crvUSDe            5.860061
autoUSD -> fUSDC             -1.156570
autoUSD -> fUSDT              3.808861
autoUSD -> frxsUSDS           2.970365
autoUSD -> gtUSDC            -0.103814
autoUSD -> gtUSDCcore        -1.699418
autoUSD -> steakUSDC          0.200760
autoUSD -> waEthUSDT          0.725906
crv2pool -> crvU-sUSDe       -2.354328
crv2pool -> crvUSDUSDC-f     -2.172473
crv2pool -> crvUSDe           2.408789
crv2pool -> fUSDC            -0.503318
crv2pool -> frxsUSDS          5.724913
crv2pool -> waEthUSDT        19.784558
crvUSDUSDT-f -> autoUSD       1.055162
crvUSDUSDT-f -> crvUSDe       0.546414
crvUSDUSDT-f -> fUSDC        -4.549465
crvUSDe -> crvUSDUSDC-f       4.567400
crvUSDe -> frxsUSDS          -0.489678
fUSDC -> GHOcrv

In [ ]:
by_move_name = df.groupby("move_name")[["spot_value_out", "real_spot_swap_cost"]].sum()
by_move_name["all_time_spot_slippage"] = 10_000 * by_move_name["real_spot_swap_cost"] / by_move_name["spot_value_out"]

px.bar(by_move_name["all_time_spot_slippage"], orientation="h").show()

px.scatter(by_move_name, x="spot_value_out", y="all_time_spot_slippage", color=by_move_name.index)